In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [29]:
my_data.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 3 rows



In [6]:
my_log_reg_model = LogisticRegression()

In [7]:
# my model is fitted to the data
fitted_logreg_model = my_log_reg_model.fit(my_data)

In [8]:
log_summary = fitted_logreg_model.summary

In [9]:
log_summary.predictions

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [10]:
# checkout the columns
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



probability - probability for the prediction value 
prediction  - the model predicted 

In [30]:
log_summary.predictions.show(3)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows



In [12]:
# vamos separar dados de treino de dados de teste
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

In [13]:
# train on lr_train, evaluate on lr_test
final_model = LogisticRegression()

In [15]:
fit_final = final_model.fit(lr_train)

In [16]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [31]:
# vamos verificar as predicoes do modelo - compare os campos label e prediction
prediction_and_labels.predictions.show(3)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[22.0320787578321...|[0.99999999972985...|       0.0|
|  0.0|(692,[122,123,148...|[20.0265727885869...|[0.99999999799289...|       0.0|
|  0.0|(692,[123,124,125...|[32.9010029233935...|[0.99999999999999...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows



In [18]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                   MulticlassClassificationEvaluator)

In [19]:
# keep defaults "label" and "rawPrediction" parameters
my_eval = BinaryClassificationEvaluator()

In [21]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [22]:
# the area under the curve ROC is 1 - perfect fit and predicted everything accurately.
my_final_roc

1.0